In [1]:
import __init__
import os
import numpy as np
import pandas as pd
import yaml
import joblib
import shap
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import roc_auc_score, roc_curve, balanced_accuracy_score
import matplotlib.pyplot as plt
from datapath_manager import DataPathManager, ITWDataPathManager
from trainers import MachineLearningModelTrainer, BranchNeuralNetworkTrainer
from dataloader import EmbeddingDataLoader
from evaluators import Evaluator
from statsmodels.stats.outliers_influence import variance_inflation_factor

/home/nmduy/anaconda3/envs/stress/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_id = 'ltkhiem'
dates = ['2022-10-16', '2022-10-17', '2022-10-18', '2022-10-20', '2022-10-21', '2022-10-22']
feature_lists = ['min_temp', 'mean_temp', 'max_temp', 'eda_slope', 'corr', 'std_HRV', 'HRV_SD2', 'HRV_RMSSD', 'std_HR', 'total_power']
# user_id = 'lzhou'
# dates = ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16']
# feature_lists = ['mean_temp', 'eda_slope', 'HRV_HTI', 'range_temp', 'nn20', 'kurtosis_relativeRRI', 'temp_slope', 'skewness_scr', 'mean_first_grad', 'num_scr_peaks']
# user_id = 'tlduyen'
# dates = ['2022-09-27', '2022-09-28', '2022-09-30', '2022-10-01', '2022-10-02', '2022-10-03']
# feature_lists = ['min_temp', 'max_temp', 'mean_temp', 'temp_slope', 'skewness_HRV', 'mean_HR', 'std_HR', 'nn20', 'eda_slope', 'rms']

In [3]:
model_path = os.path.abspath(f'./models/{user_id}')
feature_names = [line.strip() for line in open('feature_names.txt', 'r')]

In [4]:
def boxplot_by_features(feature_lists, X, Y, Z = None, T = None, labels = ['Stress', 'Relax']):
    """
    X: feature matrix
    Y: feature matrix
    X and Y should have the same number of rows
    """
    # nrows, ncols = 12, 6
    nrows, ncols = 2, 5
    fig, ax = plt.subplots(nrows, ncols, figsize=(20, 8))

    indices = [feature_names.index(feature) for feature in feature_lists]
    num_features = len(feature_lists)
    for i in range(num_features):
        data = [X[:, indices[i]], Y[:, indices[i]]]
        if Z is not None:
            if Z.shape[0] == 0:
                data.append([])
            else:
                data.append(Z[:, indices[i]])
        if T is not None:
            data.append(T[:, indices[i]])
        x, y = i // ncols, i % ncols
        ax[x, y].boxplot(data, labels=labels)
        ax[x, y].set_title(feature_lists[i])

# Lab-based model for the targeted user

In [5]:
lab_dataset_name = 'DCU_NVT_EXP2'
window_size = 60
window_shift = 0.25
signal_type = 'bvp_eda_temp'

In [6]:
dp_manager = DataPathManager(lab_dataset_name)
lab_feature_folder_path = os.path.dirname(dp_manager.get_feature_path(user_id, signal_type, window_size, window_shift))
ground_truth_path = os.path.join(lab_feature_folder_path, 'ground_truth.npy')
tasks_index_path = os.path.join(lab_feature_folder_path, 'tasks_index.npy')

In [7]:
lab_features = []
signals = ['bvp', 'eda', 'temp']
for signal in signals:
    signal_path = os.path.join(lab_feature_folder_path, f'{signal}.npy')
    signal_data = np.load(signal_path)
    lab_features.append(signal_data)
lab_features = np.concatenate(lab_features, axis=1)
tasks_index = np.load(tasks_index_path)
ground_truth = np.load(ground_truth_path) 

In [8]:
# Split train/test lab-based model
def split_train_test(indices, test_size: float = 0.3):
        """
        Split train and test data for subject-dependent model training:
            - Train_data: (1 - test_size) * number of data of a class
            - Test_data: test_size * number of data of a class 
        NOTE: This means that this approach of data splitting simulate the real-life situation 
        where the test data is the segment of data that is recorded later after we have the train data.
        """
        cut_point = int((1 - test_size) * len(indices))
        train_indices = indices[:cut_point].tolist()
        test_indices = indices[cut_point:].tolist()
        return train_indices, test_indices


TEST_SIZE = 0.2
VALID_SIZE = 0.1
indices = np.arange(lab_features.shape[0]) # The indices of the lab-based features
train_indices, valid_indices, test_indices = [], [], []
for _, task_test_index in LeaveOneGroupOut().split(indices, y=None, groups=tasks_index):
    task_train_indices, task_test_indices = split_train_test(indices[task_test_index], test_size = TEST_SIZE)
    task_train_indices, task_valid_indices = split_train_test(indices[task_train_indices], test_size = VALID_SIZE)
    train_indices += task_train_indices
    valid_indices += task_valid_indices
    test_indices += task_test_indices

### 1. Train Deep-Fusion Model

In [9]:
X_train, y_train = lab_features[train_indices], ground_truth[train_indices]
X_valid, y_valid = lab_features[valid_indices], ground_truth[valid_indices]
X_test, y_test = lab_features[test_indices], ground_truth[test_indices]

In [10]:
train_dataloader = EmbeddingDataLoader(X_train, y_train)
validate_dataloader = EmbeddingDataLoader(X_valid, y_valid)
test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [11]:
# Load deep model configuration
user_model_saved_path = os.path.join(f'{model_path}/lab_deep_fusion.pth')
config_path = os.path.join(
    os.path.dirname(os.getcwd()), 
    'models', 'model_config', 
    f'branchnn_sensor_combination_{signal_type}.yaml'
)
config_dict = yaml.safe_load(open(config_path, 'r'))

In [12]:
df_clf = BranchNeuralNetworkTrainer('./logs.txt', 
    user_model_saved_path, 
    config_dict, 
    target_metrics=['balanced_accuracy', 'precision', 'recall'],
)

LOAD PRETRAINED MODEL


In [13]:
# df_clf.train(train_dataloader, validate_dataloader, num_epochs=1000)

In [ ]:
y_pred = df_clf.predict(test_dataloader)
print(Evaluator().evaluate(y_test, y_pred))

### 2. Train ExtraTreesClassifier Model

In [ ]:
# X_train = np.concatenate([lab_features[train_indices], lab_features[valid_indices]], axis=0)
# # y_train = np.concatenate([ground_truth[train_indices], ground_truth[valid_indices]], axis=0)
# X_test, y_test = lab_features[test_indices], ground_truth[test_indices]

et_clf = ExtraTreesClassifier(
    n_estimators = 500,
    random_state = 0, 
    n_jobs = -1, 
    max_features = 'sqrt', 
    max_depth = 8, 
    min_samples_split = 2, 
    min_samples_leaf = 8,
    oob_score = True, 
    bootstrap = True, 
    class_weight = 'balanced'
)
et_clf.fit(X_train, y_train)

In [ ]:
y_pred = et_clf.predict(X_valid)
print(Evaluator().evaluate(y_valid, y_pred))

In [ ]:
y_pred = et_clf.predict(X_test)
print(Evaluator().evaluate(y_test, y_pred))

In [ ]:
# explainer = shap.TreeExplainer(et_clf)
# shap_values = explainer.shap_values(X_train)

In [ ]:
# shap.summary_plot(shap_values[1], X_train, feature_names=feature_names, plot_type='violin', max_display=10, )

In [ ]:
# shap.summary_plot(shap_values[1], X_train, feature_names=feature_names, plot_type='bar', max_display=10)

In [ ]:
data = {'model': et_clf, 'scaler': None}
joblib.dump(data, os.path.join(model_path, 'lab_et_clf.pkl'))

### 3. Train Logistic Regression Model

In [ ]:
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(np.concatenate([lab_features[train_indices], lab_features[valid_indices]], axis=0))
y_train = np.concatenate([ground_truth[train_indices], ground_truth[valid_indices]], axis=0)
X_valid, y_valid = std_scaler.transform(lab_features[valid_indices]), ground_truth[valid_indices]
X_test, y_test = std_scaler.transform(lab_features[test_indices]), ground_truth[test_indices]

lr_clf = LogisticRegression(
    random_state = 0,
    class_weight = 'balanced',
    n_jobs = -1,
    solver = 'saga',
    max_iter = 1000,
)
lr_clf.fit(X_train, y_train)

In [ ]:
y_pred = lr_clf.predict(X_train)
print(Evaluator().evaluate(y_train, y_pred))

In [ ]:
y_pred = lr_clf.predict(X_valid)
print(Evaluator().evaluate(y_valid, y_pred))

In [ ]:
y_pred = lr_clf.predict(X_test)
print(Evaluator().evaluate(y_test, y_pred))

In [ ]:
# considered_features = feature_names.copy()
# remove_features = []
# THRESHOLD = 5
# X = X_train.copy()
# while True:
#     changed = False
#     vif = pd.DataFrame()
#     indices = [feature_names.index(f) for f in considered_features]
#     _X = X[:, indices]
#     vif["VIF Factor"] = [variance_inflation_factor(_X, i) for i in range(len(indices))]
#     vif['features'] = considered_features
#     f = vif.round(1).sort_values('VIF Factor', ascending=False).iloc[0]
#     if f['VIF Factor'] > THRESHOLD:
#         remove_features.append(f['features'])
#         considered_features.remove(f['features'])
#         changed = True
#         print(f'Removing {f}')
#     if changed == False:
#         break

In [ ]:
# indices = [feature_names.index(f) for f in considered_features]
# X = X[:, indices]
# lr_clf.fit(X, y_train)

In [ ]:
# X_v = X_valid[:, indices]
# X_t = X_test[:, indices]

In [ ]:
# y_preds = lr_clf.predict(X_v)
# print(Evaluator().evaluate(y_valid, y_preds))
# y_preds = lr_clf.predict(X_t)
# print(Evaluator().evaluate(y_test, y_preds))

In [ ]:
# explainer = shap.LinearExplainer(lr_clf, X)
# shap_values = explainer.shap_values(X)

In [ ]:
# shap.summary_plot(shap_values, X, feature_names=considered_features, plot_type='violin', max_display=10)

In [ ]:
# shap.summary_plot(shap_values, X, feature_names=considered_features, plot_type='bar', max_display=10)

In [ ]:
data = {'model': lr_clf, 'scaler': std_scaler}
joblib.dump(data, os.path.join(model_path, 'lab_lr_clf.pkl'))

# In-the-wild model for the targeted user

In [14]:
itw_dataset_name = 'DCU_EXP2_ITW'

In [15]:
def get_features_and_labels(user_id: str, date: str,):
    dataset_path = ITWDataPathManager(itw_dataset_name).get_dataset_path()

    user_date_feature_path = os.path.join(dataset_path, 'features', user_id, date)
    feature_path = os.path.join(user_date_feature_path, 'X.npy')
    gt_path = os.path.join(user_date_feature_path, 'y.npy')

    feat = np.nan_to_num(np.load(feature_path))[:, :72]
    gt = np.load(gt_path)
    return feat, gt

In [16]:
def get_moments_features_and_labels(user_id: str, date: str,):
    dataset_path = ITWDataPathManager(itw_dataset_name).get_dataset_path()

    user_date_feature_path = os.path.join(dataset_path, 'features', user_id, date)
    feature_path = os.path.join(user_date_feature_path, 'X_moment.npy')
    gt_path = os.path.join(user_date_feature_path, 'y_moment.npy')

    feat = np.nan_to_num(np.load(feature_path))[:, :72]
    gt = np.load(gt_path)
    return feat, gt

In [17]:
# Load data
feat_dates = [get_features_and_labels(user_id, date) for date in dates]
for x in feat_dates:
    print(Counter(x[1]))

Counter({0.0: 7250, 1.0: 560})
Counter({0.0: 4793, 1.0: 3250})
Counter({0.0: 11270, 1.0: 2740})
Counter({0.0: 8185})
Counter({0.0: 4497, 1.0: 2460})
Counter({0.0: 2768, 1.0: 960})


In [18]:
# Load data
moments_feat_dates = [get_moments_features_and_labels(user_id, date) for date in dates]
for x in moments_feat_dates:
    print(Counter(x[1]))

Counter({0: 725, 1: 56})
Counter({0: 480, 1: 325})
Counter({0: 1127, 1: 274})
Counter({0: 819})
Counter({0: 450, 1: 246})
Counter({0: 277, 1: 96})


## 1. Apply lab-based model to in-the-wild data

In [19]:
X_test_itw = np.concatenate([x[0] for x in feat_dates], axis=0)
y_test_itw = np.concatenate([x[1] for x in feat_dates], axis=0)
X_test_moment_itw = np.concatenate([x[0] for x in moments_feat_dates], axis=0)
y_test_moment_itw = np.concatenate([x[1] for x in moments_feat_dates], axis=0)

In [20]:
# X_train_stress = X_train[y_train == 1]
# X_train_relax = X_train[y_train == 0]
# X_test_stress = X_test_moment_itw[y_test_moment_itw == 1]
# X_test_relax = X_test_moment_itw[y_test_moment_itw == 0]

In [21]:
# boxplot_by_features(feature_lists, X_train_stress, X_train_relax, X_test_stress, X_test_relax, labels = ['Train_S', 'Train_R', 'ITW_S', 'ITW_R'])

In [22]:
itw_test_dataloader = EmbeddingDataLoader(X_test_itw, y_test_itw)
itw_moment_test_dataloader = EmbeddingDataLoader(X_test_moment_itw, y_test_moment_itw)

In [23]:
itw_user_model_saved_path = os.path.abspath(f'{model_path}/itw_deep_fusion.pth')
itw_df_clf = BranchNeuralNetworkTrainer('./logs.txt',
    itw_user_model_saved_path,
    config_dict,
    target_metrics=['balanced_accuracy', 'f1'],
    pretrained_model_path = user_model_saved_path
)
itw_et_clf = joblib.load(os.path.join(model_path, 'lab_et_clf.pkl'))['model']
itw_lr_clf = joblib.load(os.path.join(model_path, 'lab_lr_clf.pkl'))['model']
std_scaler = joblib.load(os.path.join(model_path, 'lab_lr_clf.pkl'))['scaler']

LOAD PRETRAINED MODEL


In [24]:
# Lab-based Deep Fusion model applied to ITW data
print("--- Deep Fusion ---")
y_pred_itw = itw_df_clf.predict(itw_test_dataloader)
print(Evaluator().evaluate(y_test_itw, y_pred_itw))
print("--- Extra Trees ---")
y_pred_itw = itw_et_clf.predict(X_test_itw)
print(Evaluator().evaluate(y_test_itw, y_pred_itw))
print("--- Logistic Regression ---")
y_pred_itw = itw_lr_clf.predict(std_scaler.transform(X_test_itw))
print(Evaluator().evaluate(y_test_itw, y_pred_itw))

--- Deep Fusion ---
{'accuracy': 0.5466521658834875, 'balanced_accuracy': 0.5319332542942659, 'precision': 0.22736472810686817, 'recall': 0.5070210631895687, 'f1': 0.31394590566096325}
--- Extra Trees ---
{'accuracy': 0.4862413559600271, 'balanced_accuracy': 0.560825434795732, 'precision': 0.23812006813362532, 'recall': 0.6870611835506519, 'f1': 0.35366703668327437}
--- Logistic Regression ---
{'accuracy': 0.5465085260501098, 'balanced_accuracy': 0.5353446079797062, 'precision': 0.22957909755662564, 'recall': 0.5164493480441325, 'f1': 0.3178591271066115}


In [25]:
# Lab-based Deep Fusion model applied to ITW data
print("--- Deep Fusion ---")
y_pred_itw = itw_df_clf.predict(itw_moment_test_dataloader)
print(Evaluator().evaluate(y_test_moment_itw, y_pred_itw))
print("--- Extra Trees ---")
y_pred_itw = itw_et_clf.predict(X_test_moment_itw)
print(Evaluator().evaluate(y_test_moment_itw, y_pred_itw))
print("--- Logistic Regression ---")
y_pred_itw = itw_lr_clf.predict(std_scaler.transform(X_test_moment_itw))
print(Evaluator().evaluate(y_test_moment_itw, y_pred_itw))

--- Deep Fusion ---
{'accuracy': 0.5462564102564103, 'balanced_accuracy': 0.5318685556411369, 'precision': 0.22721149528513696, 'recall': 0.5075225677031093, 'f1': 0.31389578163771714}
--- Extra Trees ---
{'accuracy': 0.4867692307692308, 'balanced_accuracy': 0.5615411215596248, 'precision': 0.2384428223844282, 'recall': 0.6880641925777332, 'f1': 0.354155911202891}
--- Logistic Regression ---
{'accuracy': 0.5456410256410257, 'balanced_accuracy': 0.5370703394350147, 'precision': 0.23053097345132742, 'recall': 0.522567703109328, 'f1': 0.3199263125575683}


In [ ]:
# explainer = shap.TreeExplainer(itw_et_clf)
# shap_values = explainer.shap_values(X_test_moment_itw)

In [ ]:
# shap.summary_plot(shap_values[1], X_test_moment_itw, feature_names=feature_names, plot_type='violin', max_display=10)

In [ ]:
# shap.summary_plot(shap_values[1], X_test_moment_itw, feature_names=feature_names, plot_type='bar', max_display=10)

In [ ]:
# explainer = shap.TreeExplainer(lr_clf)
# shap_values = explainer.shap_values(std_scaler.transform(X_test_moment_itw))

## 2. Fine-tune the lab-based model to adapt to the in-the-wild data

In [ ]:
# Use the first 3 days for training
X_train = np.concatenate([x[0] for x in feat_dates[:3]])
y_train = np.concatenate([x[1] for x in feat_dates[:3]]).astype(int)
# Use the last 3 days for testing
X_test = np.concatenate([x[0] for x in feat_dates[3:]])
y_test = np.concatenate([x[1] for x in feat_dates[3:]]).astype(int)

### Fine-tune Deep-Fusion Model

In [ ]:
itw_train_dataloader = EmbeddingDataLoader(X_train, y_train)
itw_test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [ ]:
itw_user_model_saved_path = os.path.abspath(f'{model_path}/itw_deep_fusion.pth')
itw_df_clf = BranchNeuralNetworkTrainer('./logs.txt', 
    itw_user_model_saved_path, 
    config_dict, 
    target_metrics=['balanced_accuracy', 'precision', 'recall'],
)

In [ ]:
# itw_df_clf.train(itw_train_dataloader, itw_test_dataloader, num_epochs=1000)

In [ ]:
y_pred_itw = itw_df_clf.predict(itw_test_dataloader)
print(Evaluator().evaluate(y_test, y_pred_itw))

## 3. Re-train ML Model

### Re-train ExtraTreesClassifier Model

In [ ]:
itw_et_clf = ExtraTreesClassifier(
    n_estimators = 500,
    random_state = 0, 
    n_jobs = -1, 
    max_features = 'sqrt', 
    max_depth = 8, 
    min_samples_split = 2, 
    min_samples_leaf = 8,
    # oob_score = True, 
    # bootstrap = True, 
    class_weight = 'balanced'
)
itw_et_clf.fit(X_train, y_train)

In [ ]:
y_pred_itw = itw_et_clf.predict(X_test)
print(Evaluator().evaluate(y_test, y_pred_itw))

In [ ]:
data = {'model': itw_et_clf, 'scaler': None}
joblib.dump(data, os.path.join(model_path, 'itw_et_clf.pkl'))

### Re-train Logistic Regression Model

In [ ]:
scaler = StandardScaler()

In [ ]:
itw_lr_clf = LogisticRegression(
    random_state = 0,
    class_weight = 'balanced',
    n_jobs = -1,
    solver = 'saga',
    max_iter = 3000,
)
itw_lr_clf.fit(scaler.fit_transform(X_train), y_train)

In [ ]:
y_pred_itw = itw_lr_clf.predict(scaler.transform(X_test))
print(Evaluator().evaluate(y_test, y_pred_itw))

In [ ]:
data = {'model': itw_lr_clf, 'scaler': scaler}
joblib.dump(data, os.path.join(model_path, 'itw_lr_clf.pkl'))

## 4. Evaluate the in-the-wild model with lifelog image moments

In [ ]:
# Use the first 3 days for training
X_train_moments = np.concatenate([x[0] for x in moments_feat_dates[:3]])
y_train_moments = np.concatenate([x[1] for x in moments_feat_dates[:3]]).astype(int)
# Use the last 3 days for testing
X_test_moments = np.concatenate([x[0] for x in moments_feat_dates[3:]])
y_test_moments = np.concatenate([x[1] for x in moments_feat_dates[3:]]).astype(int)

In [ ]:
mitw_train_dataloader = EmbeddingDataLoader(X_train_moments, y_train_moments)
mitw_test_dataloader = EmbeddingDataLoader(X_test_moments, y_test_moments)

In [ ]:
y_train_preds = itw_df_clf.predict(mitw_train_dataloader)
print(Evaluator().evaluate(y_train_moments, y_train_preds))

In [ ]:
y_preds = itw_df_clf.predict(mitw_test_dataloader)
# y_preds = np.where(y_preds > df_thresholdOpt, 1, 0)
print(Evaluator().evaluate(y_test_moments, y_preds))

In [ ]:
y_preds = itw_et_clf.predict(X_test_moments)
# y_preds = np.where(y_preds > thresholdOpt, 1, 0)
print(Evaluator().evaluate(y_test_moments, y_preds))

In [ ]:
# explainer = shap.TreeExplainer(itw_et_clf)
# shap_values = explainer.shap_values(X_test_moments)

In [ ]:
# train_shap_values = explainer.shap_values(X_train_moments)

In [ ]:
# shap.summary_plot(train_shap_values[1], X_train_moments, feature_names=feature_names, plot_type='violin', max_display=10)

In [ ]:
# shap.summary_plot(shap_values[1], X_test_moments, feature_names=feature_names, plot_type='violin', max_display=10)

In [ ]:
y_preds = itw_lr_clf.predict(scaler.transform(X_test_moments))
# y_preds = np.where(y_preds > thresholdOpt, 1, 0)
print(Evaluator().evaluate(y_test_moments, y_preds))

In [ ]:
# explainer = shap.LinearExplainer(itw_lr_clf, scaler.transform(X_train_moments))
# shap_values = explainer.shap_values(scaler.transform(X_test_moments))

In [ ]:
# shap.summary_plot(shap_values, scaler.transform(X_test_moments), feature_names=feature_names, plot_type='bar', max_display=10)

In [ ]:
all = np.concatenate([X_train_moments, X_test_moments])
all_labels = np.concatenate([y_train_moments, y_test_moments])
mitw = EmbeddingDataLoader(all, all_labels)

In [ ]:
y_preds = itw_df_clf.predict(mitw)
print(Evaluator().evaluate(all_labels, y_preds))
y_preds = itw_et_clf.predict(all)
print(Evaluator().evaluate(all_labels, y_preds))
y_preds = itw_lr_clf.predict(scaler.transform(all))
print(Evaluator().evaluate(all_labels, y_preds))

## 5. Ensemble the models

In [ ]:
# df_preds_prob = itw_df_clf.predict_proba(mitw_test_dataloader)
# et_preds_prob = itw_et_clf.predict_proba(X_test_moments)[:, 1]

In [ ]:
# preds_prob = 0.05 * np.array(df_preds_prob) + 0.95 * np.array(et_preds_prob)
# preds = np.where(preds_prob > 0.5, 1, 0)

In [ ]:
# print(Evaluator().evaluate(y_test_moments, preds))